Exercise : Container Life-Cycle
-----------------------------------------


### Image Registry Selection (Dev + Ops)

Choose an Image Registry, in our example we use https://hub.docker.com/ again.

If not already done, create an account on https://hub.docker.com/.

And then:
* Set username and password as environment variables (as Python script)
* execute `docker login`

In [ ]:
import os
name = input("Username: ")
os.environ["USER"] = name

from getpass import getpass
password = getpass("Password: ")
os.environ["PW"] = password

In [ ]:
! docker login -u ${USER} -p ${PW}

- - -
### Creation of the image (Dev)

To do this, we first create our (PHP) microservice

In [ ]:
%%bash
cat <<%EOF% >index.php
 <?php echo '<p>Hallo Welt</p>'; ?>
%EOF%

And for the packaging of the microservices, as a container image, a Dockerfile

In [ ]:
%%bash
cat <<%EOF% >Dockerfile
FROM public.ecr.aws/docker/library/php:7.4-apache
COPY index.php /var/www/html/
%EOF%

Both together result in our container image with `docker build`.

In [ ]:
%%bash
docker build -t index.docker.io/${USER}/myservice .

The result is a new image and that Base Image (php) was downloaded:

In [ ]:
! docker image ls

- - -
### Saving the image in the registry (Dev)

When we are sure everything is running we push the container image into our registry.

In [ ]:
! docker push index.docker.io/${USER}/myservice

***
# Production (Ops)

The operation fetches the container image from the registry and starts it using Kubernetes.

In [ ]:
! kubectl run myservice --image=${USER}/myservice --restart=Never
! kubectl expose pod/myservice --type="LoadBalancer" --port 80 
! echo "http://"$(cat ~/work/server-ip)":$(kubectl get service myservice -o=jsonpath='{ .spec.ports[0].nodePort }')/"

***
Cleanup

In [ ]:
! kubectl delete pod/myservice
! kubectl delete service/myservice